<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 4 Lesson 1*

# Convolutional Neural Networks (CNNs)
## _aka_ COMPUTER VISION!

![An XKCD that hasn't quite aged so well](https://imgs.xkcd.com/comics/tasks.png)

In [0]:
from IPython.display import YouTubeVideo
YouTubeVideo('MPU2HistivI', width=600, height=400)

# Lecture

## Transfer Learning - TensorFlow Hub

"A library for reusable machine learning modules"

This lets you quickly take advantage of a model that was trained with thousands of GPU hours. It also enables transfer learning - reusing a part of a trained model (called a module) that includes weights and assets, but also training the overall model some yourself with your own data. The advantages are fairly clear - you can use less training data, have faster training, and have a model that generalizes better.

https://www.tensorflow.org/hub/

**WARNING** - Dragons ahead!

![Dragon](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Friedrich-Johann-Justin-Bertuch_Mythical-Creature-Dragon_1806.jpg/637px-Friedrich-Johann-Justin-Bertuch_Mythical-Creature-Dragon_1806.jpg)

TensorFlow Hub is very bleeding edge, and while there's a good amount of documentation out there, it's not always updated or consistent. You'll have to use your problem-solving skills if you want to use it!

In [0]:
# TF Hub landing page example
import tensorflow as tf
import tensorflow_hub as hub

with tf.Graph().as_default():
  module_url = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1"
  embed = hub.Module(module_url)
  embeddings = embed(["A long sentence.", "single-word",
                      "http://example.com"])

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())

    print(sess.run(embeddings))

What are we getting? In this case, high quality sentence embeddings, with fairly little work. There also exist pretrained networks for images, which we will use.

## Image Similarity

Let's use a pretrained neural network to calculate image similarity. The process is similar to the image embeddings from Basilica.ai, but we'll be running the inference ourselves (though not the training - training cutting edge models can cost tens of thousands of dollars in cloud GPU hours).

First pick a module to instantiate - you can [browse modules](https://www.tensorflow.org/hub/modules/) and get fully trained state-of-the-art networks for image classification, text embeddings, and others.

Let's try [DELF](https://www.tensorflow.org/hub/modules/google/delf/1), the DEep Local Features module - it is trained on photographs of landmarks, and describes input images as "noteworthy" points of vectors. This facilitates matching two similar images (e.g. two people taking a picture of the same landmark).

In [0]:
# Step 0 - Imports
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
import tensorflow as tf
import tensorflow_hub as hub

In [0]:
# Step 1 - Instantiate the module
delf_module = hub.Module("https://tfhub.dev/google/delf/1")

In [0]:
# Step 2 - Acquire images
import requests

image_urls = ["https://upload.wikimedia.org/wikipedia/commons/thumb/6/66/The_Leaning_Tower_of_Pisa_SB.jpeg/672px-The_Leaning_Tower_of_Pisa_SB.jpeg",
              "https://www.publicdomainpictures.net/pictures/120000/velka/leaning-tower-of-pisa-1427012597XXV.jpg"]

for _id,img in enumerate(image_urls): 
    r = requests.get(img)
    with open(f'tower{_id}.jpg', 'wb') as f:
        f.write(r.content)

In [0]:
# Step 3 - Transform the images for DELF
IMAGE_1_JPG = 'tower0.jpg'
IMAGE_2_JPG = 'tower1.jpg'

def resize_image(filename, new_width=256, new_height=256):
  pil_image = Image.open(filename)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert('RGB')
  pil_image_rgb.save(filename, format='JPEG', quality=90)

resize_image(IMAGE_1_JPG)
resize_image(IMAGE_2_JPG)

def show_images(image_path_list):
  plt.figure()
  for i, image_path in enumerate(image_path_list):
    plt.subplot(1, len(image_path_list), i+1)
    plt.imshow(np.asarray(Image.open(image_path)))
    plt.title(image_path)
    plt.grid(False)
    plt.yticks([])
    plt.xticks([])
  plt.show()

show_images([IMAGE_1_JPG, IMAGE_2_JPG])

In [0]:
# Step 4 - Run DELF for each image
def image_input_fn():
  filename_queue = tf.train.string_input_producer(
      [IMAGE_1_JPG, IMAGE_2_JPG], shuffle=False)
  reader = tf.WholeFileReader()
  _, value = reader.read(filename_queue)
  image_tf = tf.image.decode_jpeg(value, channels=3)
  return tf.image.convert_image_dtype(image_tf, tf.float32)

# The module operates on a single image at a time, so define a placeholder to
# feed an arbitrary image in.
image_placeholder = tf.placeholder(
    tf.float32, shape=(None, None, 3), name='input_image')

module_inputs = {
    'image': image_placeholder,
    'score_threshold': 100.0,
    'image_scales': [0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0],
    'max_feature_num': 1000,
}

module_outputs = delf_module(module_inputs, as_dict=True)

image_tf = image_input_fn()

with tf.train.MonitoredSession() as sess:
  results_dict = {}  # Stores the locations and their descriptors for each image
  for image_path in [IMAGE_1_JPG, IMAGE_2_JPG]:
    image = sess.run(image_tf)
    print('Extracting locations and descriptors from %s' % image_path)
    results_dict[image_path] = sess.run(
        [module_outputs['locations'], module_outputs['descriptors']],
        feed_dict={image_placeholder: image})

In [0]:
# Step 5 - Use the results to match the images
def match_images(results_dict, image_1_path, image_2_path):
  distance_threshold = 0.8

  # Read features.
  locations_1, descriptors_1 = results_dict[image_1_path]
  num_features_1 = locations_1.shape[0]
  print("Loaded image 1's %d features" % num_features_1)
  locations_2, descriptors_2 = results_dict[image_2_path]
  num_features_2 = locations_2.shape[0]
  print("Loaded image 2's %d features" % num_features_2)

  # Find nearest-neighbor matches using a KD tree.
  d1_tree = cKDTree(descriptors_1)
  _, indices = d1_tree.query(
      descriptors_2, distance_upper_bound=distance_threshold)

  # Select feature locations for putative matches.
  locations_2_to_use = np.array([
      locations_2[i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])
  locations_1_to_use = np.array([
      locations_1[indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
  ])

  # Perform geometric verification using RANSAC.
  _, inliers = ransac(
      (locations_1_to_use, locations_2_to_use),
      AffineTransform,
      min_samples=3,
      residual_threshold=20,
      max_trials=1000)

  print('Found %d inliers' % sum(inliers))

  # Visualize correspondences.
  _, ax = plt.subplots(figsize=(9, 18))
  img_1 = mpimg.imread(image_1_path)
  img_2 = mpimg.imread(image_2_path)
  inlier_idxs = np.nonzero(inliers)[0]
  plot_matches(
      ax,
      img_1,
      img_2,
      locations_1_to_use,
      locations_2_to_use,
      np.column_stack((inlier_idxs, inlier_idxs)),
      matches_color='b')
  ax.axis('off')
  ax.set_title('DELF correspondences')

match_images(results_dict, IMAGE_1_JPG, IMAGE_2_JPG)

This lets us visualize a little bit better what it means to e.g. calculate the "embeddings" of an image, and then compare the values to embeddings of another image in order to calculate similarity.

## Convolutional Neural Networks

Like neural networks themselves, CNNs are inspired by biology - specifically, the receptive fields of the visual cortex.

In [0]:
from IPython.display import YouTubeVideo
YouTubeVideo('IOHayh06LJ4', width=600, height=400)

Put roughly, in a real brain the neurons in the visual cortex *specialize* to be receptive to certain regions, shapes, colors, orientations, and other common visual features. In a sense, the very structure of our cognitive system transforms raw visual input, and sends it to neurons that specialize in handling particular subsets of it.

CNNs imitate this approach by applying a convolution. A convolution is an operation on two functions that produces a third function, showing how one function modifies another. Convolutions have a [variety of nice mathematical properties](https://en.wikipedia.org/wiki/Convolution#Properties) - commutativity, associativity, distributivity, and more. Applying a convolution effectively transforms the "shape" of the input.

One common confusion - the term "convolution" is used to refer to both the process of computing the third (joint) function and the process of applying it. In our context, it's more useful to think of it as an application, again loosely analogous to the mapping from visual field to receptive areas of the cortex in a real animal.

### Convolution - an example

Consider blurring an image - assume the image is represented as a matrix of numbers, where each number corresponds to the color value of a pixel.


In [0]:
import imageio
import matplotlib.pyplot as plt
from skimage import color, io
from skimage.exposure import rescale_intensity

austen = io.imread('https://dl.airtable.com/S1InFmIhQBypHBL0BICi_austen.jpg')
austen_grayscale = rescale_intensity(color.rgb2gray(austen))
austen_grayscale

In [0]:
austen_grayscale.shape

In [0]:
plt.imshow(austen_grayscale, cmap="gray");

One way to blur would be to replace each point with the average of itself and its neighbors.

In [0]:
austen_blur1 = austen_grayscale.copy()

for i, row in enumerate(austen_grayscale):
  for j, col in enumerate(row):
    # Bit of logic to handle edges - only doing direct non-diag neighbors
    blur_val = col
    num_averaged = 1
    if i > 0:
      blur_val += austen_grayscale[i - 1][j]
      num_averaged += 1
    if j > 0:
      blur_val += austen_grayscale[i][j - 1]
      num_averaged += 1
    if i < len(austen_grayscale) - 1:
      blur_val += austen_grayscale[i + 1][j]
      num_averaged += 1
    if j < len(row) - 1:
      blur_val += austen_grayscale[i][j + 1]
      num_averaged += 1
    austen_blur1[i][j] = blur_val / num_averaged

austen_blur1

In [0]:
plt.imshow(austen_blur1, cmap="gray");

What if we wanted to blur by averaging a larger neighborhood of pixels, but treating the closer neighbors as more important than the far ones?

One natural approach would be to use a two dimensional Normal distribution to determine the appropriate pixel weights for averaging.

In [0]:
from skimage.filters import gaussian

# Using relatively large sigma so the filter impact is clear
austen_blur2 = gaussian(austen_grayscale, sigma=5)
austen_blur2

In [0]:
plt.imshow(austen_blur2, cmap="gray")

This is called a *Gaussian blur*, and is an early and well-established application of convolutions.

![Gaussian blur applied to newspaper image](https://upload.wikimedia.org/wikipedia/commons/d/d7/Halftone%2C_Gaussian_Blur.jpg)

Also - it is a **convolution**!

Specifically, it's the application of the two dimensional Gaussian function to the function that is the image itself (something that takes values $x, y$ and returns a number for the value of that pixel).

And that's really it - there are of course many more convolutions out there. Following is a relatively simple form of edge detection as a convolution:

In [0]:
import scipy.ndimage as nd

edge_convolution = np.array([[1,1,1,1,1],
                             [0,0,0,0,0],
                             [0,0,0,0,0],
                             [0,0,0,0,0],
                             [-1,-1,-1,-1,-1]])

austen_edges = nd.convolve(austen_grayscale, edge_convolution)
austen_edges

In [0]:
plt.imshow(austen_edges, cmap="gray");

## A Typical CNN

![A Typical CNN](https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Typical_cnn.png/800px-Typical_cnn.png)

The first stage of a CNN is, unsurprisingly, a convolution - specifically, a transformation that maps regions of the input image to neurons responsible for receiving them. The convolutional layer can be visualized as follows:

![Convolutional layer](https://upload.wikimedia.org/wikipedia/commons/6/68/Conv_layer.png)

The red represents the original input image, and the blue the neurons that correspond.

As shown in the first image, a CNN can have multiple rounds of convolutions, [downsampling](https://en.wikipedia.org/wiki/Downsampling_(signal_processing)) (a digital signal processing technique that effectively reduces the information by passing through a filter), and then eventually a fully connected neural network and output layer. Typical output layers for a CNN would be oriented towards classification or detection problems - e.g. "does this picture contain a cat, a dog, or some other animal?"

Why are CNNs so popular?

1. They work, really well (see XKCD at top)
2. Compared to prior image learning techniques, they require relatively little image preprocessing (cropping/centering, normalizing, etc.)
3. Relatedly, they are *robust* to all sorts of common problems in images (shifts, lighting, etc.)

Actually training a cutting edge image classification CNN is nontrivial computationally - the good news is, with transfer learning, we can get one "off-the-shelf"!

In [0]:
import numpy as np

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def img_contains_banana(img):
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model = ResNet50(weights='imagenet')
  features = model.predict(x)
  results = decode_predictions(features, top=3)[0]
  print(results)
  for entry in results:
    if entry[1] == 'banana':
      return entry[2]
  return 0.0

In [0]:
image_urls = ["https://github.com/LambdaSchool/ML-YouOnlyLookOnce/raw/master/sample_data/negative_examples/example11.jpeg",
              "https://github.com/LambdaSchool/ML-YouOnlyLookOnce/raw/master/sample_data/positive_examples/example0.jpeg"]

for _id,img in enumerate(image_urls): 
    r = requests.get(img)
    with open(f'example{_id}.jpg', 'wb') as f:
        f.write(r.content)

In [0]:
from IPython.display import Image
Image(filename='example0.jpeg', width=600)

In [0]:
img_contains_banana(process_img_path('example11.jpeg'))

In [0]:
Image(filename='example0.jpeg', width=600)

In [0]:
img_contains_banana(process_img_path('example0.jpeg'))

Notice that, while it gets it right, the confidence for the banana image is fairly low. That's because so much of the image is "not-banana"! How can this be improved? Bounding boxes to center on items of interest (see extended resources).

# Assignment

Load a pretrained network from TensorFlow Hub, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
module = hub.Module("https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1")
height, width = hub.get_expected_image_size(module)
images = ...  # A batch of images with shape [batch_size, height, width, 3].
logits = module(images)  # Logits with shape [batch_size, num_classes].
```

Apply it to classify the images downloaded below (images from a search for animals in national parks):

In [0]:
!pip install google_images_download

In [0]:
from google_images_download import google_images_download

response = google_images_download.googleimagesdownload()
arguments = {"keywords": "animal national park", "limit": 20,
             "print_urls": True}
absolute_image_paths = response.download(arguments)


Item no.: 1 --> Item name = animal national park
Evaluating...
Starting Download...
Image URL: https://i.ytimg.com/vi/P8NJa_YoRxk/maxresdefault.jpg
Completed Image ====> 1.maxresdefault.jpg
Image URL: https://k6u8v6y8.stackpathcdn.com/blog/wp-content/uploads/2017/06/Royal-Bengal-Tiger.jpg
Completed Image ====> 2.Royal-Bengal-Tiger.jpg
Image URL: https://npca.s3.amazonaws.com/images/8135/2c7e0d75-c7ff-4336-99d7-259448d03a4d-banner.jpg?1445969501
Completed Image ====> 3.2c7e0d75-c7ff-4336-99d7-259448d03a4d-banner.jpg
Image URL: https://k6u8v6y8.stackpathcdn.com/blog/wp-content/uploads/2014/04/national-parks-and-wildlife-sanctuaries-in-india.png
Completed Image ====> 4.national-parks-and-wildlife-sanctuaries-in-india.png
Image URL: https://www.nps.gov/arch/learn/nature/images/ARK_6.jpg?maxwidth=1200&maxheight=1200&autorotate=false
Completed Image ====> 5.ARK_6.jpg
Image URL: http://www.corbettnationalpark.in/blog/wp-content/uploads/2015/08/cropped-13625772024_1fd7467d29_k1-1024x682.jpg
C

In [0]:
absolute_image_paths

({'animal national park': ['/content/downloads/animal national park/1.maxresdefault.jpg',
   '/content/downloads/animal national park/2.Royal-Bengal-Tiger.jpg',
   '/content/downloads/animal national park/3.2c7e0d75-c7ff-4336-99d7-259448d03a4d-banner.jpg',
   '/content/downloads/animal national park/4.national-parks-and-wildlife-sanctuaries-in-india.png',
   '/content/downloads/animal national park/5.ARK_6.jpg',
   '/content/downloads/animal national park/6.cropped-13625772024_1fd7467d29_k1-1024x682.jpg',
   '/content/downloads/animal national park/7.VOYA_web_deer.jpg',
   '/content/downloads/animal national park/8.a2d539ed-8489-4eb4-a135-14e7e9e0e84a-banner.jpg',
   '/content/downloads/animal national park/9.addo-elephant-national-park-elephants-590x390.jpg',
   '/content/downloads/animal national park/10.zebras-in-kidepo-750x450.jpg',
   '/content/downloads/animal national park/11.Nairobi_National_Park%2C_Kenya_%2832570316676%29.jpg',
   '/content/downloads/animal national park/12.20

Report both the most likely estimated class for any image, and also investigate (a) images where the classifier isn't that certain (the best estimate is low), and (b) images where the classifier fails.

Answer (in writing in the notebook) the following - "What sorts of images do CNN classifiers do well with? What sorts do they not do so well? And what are your hypotheses for why?"

In [0]:
def preprocess_images(img):
    decoded = tf.image.decode_jpeg(img)
    converted = tf.image.convert_image_dtype(decoded, tf.float32)
    reshaped = tf.reshape(converted, [1, 224, 224, 3])
    return reshaped

In [0]:
from keras import backend

In [0]:
from keras.backend import tf

In [0]:
ready_image = preprocess_images('/content/downloads/animal national park/2.Royal-Bengal-Tiger.jpg')

In [0]:
import tensorflow_hub as hub
module = hub.Module("https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1")
height, width = hub.get_expected_image_size(module)
images = ready_image  # A batch of images with shape [batch_size, height, width, 3].
logits = module(images)  # Logits with shape [batch_size, num_classes].

In [0]:
outputs = module(dict(images=images), signature="image_classification",
                   as_dict=True)
logits = outputs["default"]
logits

<tf.Tensor 'module_apply_image_classification/resnet_v1_50/SpatialSqueeze:0' shape=(1, 1001) dtype=float32>

In [0]:
import numpy as np
predicted_class = np.argmax(logits, axis=-1)
predicted_class

0

In [0]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

16384/10484 [==============================================] - 0s 0us/step


In [0]:
predicted_class_name = imagenet_labels[predicted_class]
predicted_class_name

'background'

In [0]:
from PIL import Image

Image(filename='/content/downloads/animal national park/2.Royal-Bengal-Tiger.jpg', width=600)

TypeError: ignored

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language